# Import

In [ ]:
import os

!pip install easyocr
import easyocr

from pathlib import Path

!pip install pdf2image
from pdf2image import convert_from_path

!apt-get install python-dev libxml2-dev libxslt1-dev antiword unrtf poppler-utils pstotext tesseract-ocr \
flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig
!pip install textract
import textract

!pip install --user -U nltk
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk import WordNetLemmatizer
import pandas as pd


!pip install tika
from tika import parser

!pip install yake
import yake

from sklearn.feature_extraction.text import TfidfVectorizer

# Functions

In [ ]:
def main(type=1):
    '''if tupe = 1, works with file. Otherwise with folder'''
    if type == 1:
      file_path = input("Enter a file path: ")
      print(text_recognition(file_path=file_path))
    else:
      folder_path = input("Enter a folder path: ")
      names = os.listdir(folder_path)
      for name in names:
        file_path = folder_path + '/' + name
        print(text_recognition(file_path=file_path)) 

In [ ]:
def text_recognition(file_path):
    '''For recognition pdf and convert to txt format. Also  convert doc/docx formats to txt'''
  
    text_file_name = f'result_{Path(file_path).stem}.txt' #name
    
    result = []
    new_text = ''

    if Path(file_path).suffix == '.pdf':
      reader = easyocr.Reader(["ru", "en"]) #the best way: many pdf docs are scans with bad resolution and without text layers
      images = convert_from_path(file_path, 500)
      for i in range(len(images)):
        images[i].save(f'out{i}.jpg', 'JPEG')
        result.append(reader.readtext(f'out{i}.jpg', detail=0, paragraph=True))
        os.remove(f'out{i}.jpg')
      for i in range(len(result)):
        for j in range(len(result[i])):
          new_text = new_text+result[i][j]
      text = text_lemma(text_preparation(new_text))

      with open(text_file_name, "w") as file:
        file.write(" ".join(text))
      
    elif Path(file_path).suffix == '.doc':
      parsed = parser.from_file(file_path)
      result.append(parsed.get('content'))
      text = text_lemma(text_preparation(result[0]))

      with open(text_file_name, "w") as file:
        file.write(" ".join(text))

    elif Path(file_path).suffix == '.docx':
      text = textract.process(file_path, extension='docx').decode('utf-8')
      text = text_lemma(text_preparation(text))
      with open(text_file_name, "w") as file:
        file.write(" ".join(text))

    else:
      pass

    try:
      df_creator(os.path.dirname(file_path)+'/'+text_file_name)
    except:
      pass

    return f"Result wrote into {text_file_name}"

In [ ]:
def text_preparation(text):  
    clean_text = text.lower().replace('_', '').replace('\n', '').replace('(', '').replace(')', '')\
    .replace("'", '').replace('"', '').replace(',', '').replace('.', '').replace('-', '')\
    .replace('%', '').replace('№', '').replace('nг\xa0  \xa0\xa0г', '').replace('1', '')\
    .replace('2', '').replace('3', '').replace('4', '').replace('5', '').replace('6', '')\
    .replace('7', '').replace('8', '').replace('9', '').replace('0', '').replace(':', '')\
    .replace(' п ', '').replace(';', '')
    return  clean_text

In [ ]:
def keyword(file_path):
  with open(file_path, "r") as file:
    text = file.read()
  extractor = yake.KeywordExtractor (
    lan = "ru",     # язык
    n = 2,          # максимальное количество слов в фразе
    dedupLim = 0.9, # порог похожести слов
    top = 10        # количество ключевых слов
  )
  keywords = []
  for i in range(len(extractor.extract_keywords(text))):
    keywords.append(extractor.extract_keywords(text)[i][0])
  
  return " ".join(keywords)

In [ ]:
def text_lemma(text):
  result =  filtr(lemmatize(tokeniser(text)))
  return result

In [ ]:
def filtr(tokens):
    stop_words = stopwords.words("russian")
    filtered_tokens = []
    for token in tokens:
      if token not in stop_words:
          filtered_tokens.append(token)
    return filtered_tokens

In [ ]:
def lemmatize(tokens):
    try:
        wnl = nltk.WordNetLemmatizer()
    except LookupError:
        nltk.download('wordnet')
        wnl = nltk.WordNetLemmatizer()
    return [wnl.lemmatize(t) for t in tokens] 

In [ ]:
def tokeniser(text):
  tokens = word_tokenize(text, language="russian")
  return tokens

In [ ]:
def df_creator(path):
  with open(path, "r") as file:
    file_text = file.read()
  df.loc[ len(df.index )] = [Path(path).name.split('_')[-1], file_text, keyword(path), 0]
  return df

# Work process


In [ ]:
df = pd.DataFrame(columns=['name', 'text', 'keywords', 'type'])

In [ ]:
main(type=0)